In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold 
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Read the cvs file
dataset = pd.read_csv('Training.csv')

# Print the shape of dataset
print(dataset.shape)

# Print all the rows of dataset
dataset.head()

(68528, 7)


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,7.97698,4.33494,10.67282,1.76692,3.22440,51.681460,3.65434
1,8.07824,4.44616,10.56160,1.70716,3.32566,51.563598,3.47672
2,8.02844,4.22372,10.56160,1.64906,3.17460,50.863080,3.47672
3,8.02844,4.22372,10.56160,1.70716,3.17460,45.841581,3.47672
4,7.87572,4.44616,10.45038,1.70716,3.27586,47.126421,3.47672


In [ ]:
# Build sequence 
def build_sequences(df, target_labels, window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))

        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
# Window size
window = 300
# Stride size
stride = 20

In [ ]:
target_labels = dataset.columns
# Telescope size
telescope = 50

In [ ]:
X, y = build_sequences(dataset, target_labels, window, stride, telescope)
X.shape, y.shape

((3418, 300, 7), (3418, 50, 7))

In [ ]:
# Apply 5-fold cross validation
kf = KFold(n_splits=5, random_state=None, shuffle=False)
indexes = kf.split(X,y)
for train_index, test_index in indexes:
    print("TRAIN:", train_index[:10], "TEST:", test_index[:10])
    print("TRAIN:", train_index.shape, "TEST:", test_index.shape)

TRAIN: [684 685 686 687 688 689 690 691 692 693] TEST: [0 1 2 3 4 5 6 7 8 9]
TRAIN: (2734,) TEST: (684,)
TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [684 685 686 687 688 689 690 691 692 693]
TRAIN: (2734,) TEST: (684,)
TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [1368 1369 1370 1371 1372 1373 1374 1375 1376 1377]
TRAIN: (2734,) TEST: (684,)
TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [2052 2053 2054 2055 2056 2057 2058 2059 2060 2061]
TRAIN: (2735,) TEST: (683,)
TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [2735 2736 2737 2738 2739 2740 2741 2742 2743 2744]
TRAIN: (2735,) TEST: (683,)


In [ ]:
# Set input shape
input_shape = X.shape[1:]
# Set output shape
output_shape = y.shape[1:]
# Set the batch size
batch_size = 64
# Set the number of epochs for training
epochs = 200

In [ ]:
# Build the Bidirectional recurrent neural network
def build_CONV_Bidirectional_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)

    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae', 'mse'])

    # Return the model
    return model

In [ ]:
mse = []
mae = []
models = []

In [ ]:
k = 0
for train_index, test_index in kf.split(X,y):
  k = k + 1
  # Instantiate the neural network
  model = build_CONV_Bidirectional_model(input_shape, output_shape)
  
  # Train the model
  history = model.fit(
      x = X[train_index],
      y = y[train_index],
      batch_size = batch_size,
      epochs = epochs,
      validation_split=.1,
      callbacks = [
          # Early Stopping
          tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
          # Method to reduce learning rate when a metric has stopped improving
          tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
      ]
  ).history

  # Start the prediction
  predictions = model.predict(X[test_index])
  
  # Print mean squared error for each step
  mean_squared_error = tfk.metrics.mse(y[test_index].flatten(),predictions.flatten())
  
  # Print mean absolute error for each step
  mean_absolute_error = tfk.metrics.mae(y[test_index].flatten(),predictions.flatten())
  
  mse.append(mean_squared_error) 
  mae.append(mean_absolute_error)
  models.append(model)
  
  # Save the five models, one for each step of cross validation. Only the model with 
  # the best rmse will be submitted in Codalab. We notice that the best model 
  # gave a better perfomance on Codalab then the one retrained on the entire dataset.
  model.save('Bidirectional_tel50_ver' + str(k))



Epoch 1/200
39/39 [==============================] - 19s 303ms/step - loss: 98.6793 - mae: 7.4429 - mse: 98.6793 - val_loss: 79.3723 - val_mae: 6.2819 - val_mse: 79.3723 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 9s 223ms/step - loss: 68.6954 - mae: 5.9103 - mse: 68.6954 - val_loss: 69.5924 - val_mae: 5.4330 - val_mse: 69.5924 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 9s 222ms/step - loss: 59.1840 - mae: 5.1250 - mse: 59.1840 - val_loss: 65.0043 - val_mae: 4.8765 - val_mse: 65.0043 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 9s 223ms/step - loss: 55.1162 - mae: 4.7535 - mse: 55.1162 - val_loss: 64.2620 - val_mae: 4.7764 - val_mse: 64.2620 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 9s 223ms/step - loss: 51.6480 - mae: 4.4823 - mse: 51.6480 - val_loss: 64.1142 - val_mae: 4.7985 - val_mse: 64.1142 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 9s 222ms/step - loss: 48.6552 - mae:

Epoch 1/200
39/39 [==============================] - 17s 274ms/step - loss: 194.4276 - mae: 10.6589 - mse: 194.4276 - val_loss: 130.8956 - val_mae: 9.7766 - val_mse: 130.8956 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 9s 223ms/step - loss: 135.8594 - mae: 9.3032 - mse: 135.8594 - val_loss: 100.8351 - val_mae: 7.8861 - val_mse: 100.8351 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 9s 218ms/step - loss: 88.1662 - mae: 6.9090 - mse: 88.1662 - val_loss: 80.0136 - val_mae: 6.1525 - val_mse: 80.0136 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 9s 219ms/step - loss: 61.9210 - mae: 5.2954 - mse: 61.9210 - val_loss: 71.8943 - val_mae: 5.5793 - val_mse: 71.8943 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 8s 218ms/step - loss: 50.4010 - mae: 4.5751 - mse: 50.4010 - val_loss: 68.5426 - val_mae: 5.2228 - val_mse: 68.5426 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 9s 219ms/step - loss: 44.23

Epoch 1/200
39/39 [==============================] - 18s 276ms/step - loss: 162.0054 - mae: 9.3018 - mse: 162.0054 - val_loss: 111.5579 - val_mae: 7.7493 - val_mse: 111.5579 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 9s 227ms/step - loss: 101.4950 - mae: 7.5645 - mse: 101.4951 - val_loss: 90.7514 - val_mae: 6.6788 - val_mse: 90.7514 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 9s 225ms/step - loss: 63.2126 - mae: 5.6859 - mse: 63.2126 - val_loss: 74.8160 - val_mae: 5.6767 - val_mse: 74.8160 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 9s 225ms/step - loss: 45.8343 - mae: 4.5628 - mse: 45.8343 - val_loss: 67.9946 - val_mae: 5.1854 - val_mse: 67.9946 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 9s 222ms/step - loss: 40.0712 - mae: 4.1163 - mse: 40.0712 - val_loss: 66.3936 - val_mae: 5.0329 - val_mse: 66.3936 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 9s 222ms/step - loss: 37.8812 

Epoch 1/200
39/39 [==============================] - 17s 276ms/step - loss: 149.7435 - mae: 8.5851 - mse: 149.7434 - val_loss: 73.0665 - val_mae: 5.6622 - val_mse: 73.0665 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 8s 208ms/step - loss: 64.5096 - mae: 5.6519 - mse: 64.5096 - val_loss: 66.9689 - val_mae: 5.1680 - val_mse: 66.9689 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 8s 208ms/step - loss: 48.4043 - mae: 4.6440 - mse: 48.4043 - val_loss: 65.8563 - val_mae: 5.0998 - val_mse: 65.8563 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 8s 207ms/step - loss: 43.8026 - mae: 4.3391 - mse: 43.8026 - val_loss: 66.0531 - val_mae: 5.0594 - val_mse: 66.0531 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 8s 207ms/step - loss: 41.5347 - mae: 4.1768 - mse: 41.5347 - val_loss: 65.8666 - val_mae: 5.1132 - val_mse: 65.8666 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 8s 206ms/step - loss: 40.4100 - ma

Epoch 1/200
39/39 [==============================] - 17s 277ms/step - loss: 185.7844 - mae: 9.9982 - mse: 185.7844 - val_loss: 108.8760 - val_mae: 7.7634 - val_mse: 108.8760 - lr: 0.0010
Epoch 2/200
39/39 [==============================] - 8s 218ms/step - loss: 145.0207 - mae: 9.2993 - mse: 145.0207 - val_loss: 104.0681 - val_mae: 7.5296 - val_mse: 104.0681 - lr: 0.0010
Epoch 3/200
39/39 [==============================] - 9s 220ms/step - loss: 120.1283 - mae: 8.3830 - mse: 120.1283 - val_loss: 98.9158 - val_mae: 7.3484 - val_mse: 98.9158 - lr: 0.0010
Epoch 4/200
39/39 [==============================] - 8s 214ms/step - loss: 98.4180 - mae: 7.4037 - mse: 98.4180 - val_loss: 94.7448 - val_mae: 7.2151 - val_mse: 94.7448 - lr: 0.0010
Epoch 5/200
39/39 [==============================] - 8s 212ms/step - loss: 80.6394 - mae: 6.5402 - mse: 80.6394 - val_loss: 90.8334 - val_mae: 7.2049 - val_mse: 90.8334 - lr: 0.0010
Epoch 6/200
39/39 [==============================] - 8s 214ms/step - loss: 67.4

In [ ]:
# Cross validation, with this data, is pessimistically biased. Indeed the rmse score 
# given by this fit is worse than the score given by submission in Codelab.

# Print the mean squared error for each model
for m1 in mse:
  print(m1.numpy())
# Print the mean absolute error for each model
for m2 in mae:
  print(m2.numpy())


55.298748
19.436071
35.455498
22.377926
40.232815
4.796805
2.8483741
3.6275663
2.6400597
3.7797348


In [ ]:
# Compute the mean of mse for the models
mse_cv = tf.math.reduce_mean(mse)

# Compute the mean of rmse for the models
rmse_cv = tf.math.sqrt(mse_cv.numpy())

# Compute the mean of mae for the models
mae_cv = tf.math.reduce_mean(mae)

print('mse average: ', mse_cv.numpy())
print('rmse average: ', rmse_cv.numpy())
print('mae average: ', mae_cv.numpy())

mse average:  34.56021
rmse average:  5.8787937
mae average:  3.538508


### Model Ensemble

During this phase we also tried a Model Ensemble approach. We started by increasing the window size, searching for a good score on the train set and a poor one on the test phase. Once obtained this overfitting models we averaged the 5 predictions. This approach gave us worse results then the single model, probably because the models were still too similar to each other.